## Table A3: Count of schools' financier alumni in Forbes 400 by schools' rejection rates

### From Elite Embeddedness

By Charlie Eaton and Albina Gibadullina

In [1]:
use data/d_school_level_elite_embed.dta, clear
keep if year==2003 | year==2017
set scheme plotplain

(795 observations deleted)


In [2]:
sum rejectrate if schooltypenum==3 & year==2003


    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
  rejectrate |         26    70.96526    15.99386   24.67949   90.01763


In [3]:
gen typeyear2=3 if schooltypenum==2 & year==2003
replace typeyear2=1 if schooltypenum==3 & year==2003
replace typeyear2=4 if schooltypenum==2 & year==2017
replace typeyear2=2 if schooltypenum==3 & year==2017
label define typeyear2 3 "Public 2003" 1 "Private 2003" ///
4 "Public 2017" 2 "Private 2017"
label val typeyear2 typeyear2

qui gen pctunderfirst= tefraceunder/teftotlt

bysort schooltypenum: sum pctunderfirst

qui gen endow_assets_c_log=log(endowment_assets / cpi_scalar_2017)
qui gen logtotal03_revenue=log(total03_revenue / cpi_scalar_2017)
qui gen logfte_count=log(teftotlt)

qui egen boardcount=rowtotal(pehedge otherfinance nonfinance)
qui gen pctalt= pehedge/boardcount * 100

(86 missing values generated)
(26 real changes made)
(30 real changes made)
(30 real changes made)

-------------------------------------------------------------------------------
-> schooltypenum = public

    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
pctunderfi~t |         59    .1639691    .0798734   .0183983   .3861746

-------------------------------------------------------------------------------
-> schooltypenum = private

    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
pctunderfi~t |         56    .1874364    .0638645   .0753589   .3296893



In [4]:
*gen rejectrate=(1-acceptrate)*100
label var rejectrate "admissions selectivity (rejection rate)"
label var endow_assets_c_log "Endowment assets (log 2017 $)"
label var logtotal03_revenue "University revenue (log 2017 $)"
label var logfte_count "FTE students (log)"
label var fgrnt_p "% students low-income"
label var pctunderfirst "% students racially underrepresented"

In [5]:
est clear
qui eststo: reg alumnipehedge rejectrate if schooltypenum==3 & year==2017, cluster(unitid)
reg alumnipehedge rejectrate if schooltypenum==3 & year==2003, cluster(unitid)

qui eststo: reg alumnipehedge 3.schooltypenum##c.rejectrate ///
    logfte_count endow_assets_c_log logtotal03_revenue pctunderfirst fgrnt_p if year==2003, cluster(unitid)

qui eststo: reg alumnipehedge 3.schooltypenum##c.rejectrate ///
    logfte_count endow_assets_c_log logtotal03_revenue pctunderfirst fgrnt_p if year==2017, cluster(unitid)


Linear regression                               Number of obs     =         26
                                                F(1, 25)          =       6.94
                                                Prob > F          =     0.0142
                                                R-squared         =     0.2617
                                                Root MSE          =     1.9303

                                (Std. err. adjusted for 26 clusters in unitid)
------------------------------------------------------------------------------
             |               Robust
alumnipehe~e | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
  rejectrate |   .0703963   .0267146     2.64   0.014     .0153765    .1254161
       _cons |  -3.726461   1.711547    -2.18   0.039    -7.251459    -.201463
------------------------------------------------------------------------------


In [6]:
est clear
qui eststo: reg alumnipehedge rejectrate if schooltypenum==3 & year==2003, cluster(unitid)

qui eststo: reg alumnipehedge rejectrate if schooltypenum==3 & year==2017, cluster(unitid)

qui eststo: reg alumnipehedge rejectrate if schooltypenum==2 & year==2003, cluster(unitid)

qui eststo: reg alumnipehedge rejectrate if schooltypenum==2 & year==2017, cluster(unitid)

In [7]:
set line 150
esttab, type cells(b(star fmt(3)) se(par fmt(3))) stardetach ///
label collabels(none) ///
starlevels(^ .1 * .05 ** .01 *** .001) eqlabels(none) ///
keep(rejectrate) ///
stats(r2 r2_within N_clust N, fmt(2 2 0 0) label(R-square Schools N)) ///
mlabel("Private 2003" "Private 2017" "Public 2003" "Public 2017" "Panel Fixed Effects" )


----------------------------------------------------------------------------------------
                              (1)              (2)              (3)              (4)    
                     Private 2003     Private 2017      Public 2003      Public 2017    
----------------------------------------------------------------------------------------
admissions selecti~o        0.070 *          0.202 *          0.036 ^          0.021 *  
                          (0.027)          (0.076)          (0.018)          (0.009)    
----------------------------------------------------------------------------------------
R-square                     0.26             0.19             0.35             0.29    
Schools                                                                                 
N                              26               30               29               30    
N                              26               30               29               30    
--------------------

In [8]:
qui esttab using tables/ta3_forbes400_admissions.rtf, ///
type cells(b(star fmt(3)) se(par fmt(3))) stardetach ///
label collabels(none) ///
starlevels(^ .1 * .05 ** .01 *** .001) eqlabels(none) ///
keep(rejectrate) ///
stats(r2 r2_within N_clust N, fmt(2 2 0 0) label(R-square Schools N)) ///
mlabel("Private 2003" "Private 2017" "Public 2003" "Public 2017" "Panel Fixed Effects" )

file tables/ta3_forbes400_admissions.rtf already exists
r(602);


In [ ]:
gsort year -schooltypenum schoolname
list schoolname rejectrate alumnipehedge unitid if year==2017 & schooltypenum==3

In [ ]:
gsort year -schooltypenum schoolname
list schoolname rejectrate alumnipehedge unitid if year==2003 & schooltypenum==3

In [ ]:
set line 200
gsort -year schooltypenum schoolname
list schoolname rejectrate alumnipehedge unitid if year==2017 & schooltypenum==2

In [ ]:
set line 200
gsort year schooltypenum schoolname
list schoolname rejectrate alumnipehedge unitid if year==2003 & schooltypenum==2